In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [ ]:
CApatient = pd.read_csv('mimic CA.csv')
CApatient

,stay_id,subject_id,gender,anchor_age,starttime,charttime,heart_rate,sbp_ni,dbp_ni,resp_rate,temperature,spo2
0,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:07:00,114.0,NaN,NaN,18.0,NaN,NaN
1,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:18:00,NaN,NaN,NaN,NaN,NaN,96.0
2,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 18:32:00,108.0,NaN,NaN,20.0,NaN,95.0
3,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:00:00,111.0,NaN,NaN,18.0,NaN,96.0
4,30031755,19867017,F,74,2124-04-24 17:00:00,2124-04-19 20:15:00,NaN,85.0,60.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
68968,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 06:40:00,85.0,NaN,NaN,11.0,NaN,99.0
68969,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 07:00:00,95.0,NaN,NaN,16.0,NaN,99.0
68970,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 08:00:00,100.0,NaN,NaN,18.0,37.833333,99.0
68971,39992578,19774838,M,69,2187-09-15 10:00:00,2187-09-15 09:00:00,104.0,NaN,NaN,22.0,NaN,98.0


In [ ]:
# 列出每位CA患者的性別和年齡作為配對依據
CA1 = CApatient[['subject_id', 'gender', 'anchor_age']]
CA = CA1.drop_duplicates()
CA

,subject_id,gender,anchor_age
0,19867017,F,74
188,11279605,M,87
323,15896656,M,72
328,18756147,M,41
488,13662907,F,35
...,...,...,...
68784,10143305,M,56
68811,14559749,M,71
68825,14533062,F,73
68922,12569468,M,56


In [ ]:
# 所有nonCA的患者id
noCA = pd.read_csv('nonCA patient.csv')
noCA

,subject_id,gender,anchor_age
0,10000032,F,52
1,10000048,F,23
2,10000084,M,72
3,10000102,F,27
4,10000108,M,25
...,...,...,...
279367,19999828,F,46
279368,19999829,F,28
279369,19999840,M,58
279370,19999914,F,49


In [ ]:
# 因為chartevent是發生在ICU的患者資料，所以不能直接從noCA的table配對患者，因此先在chartevents.csv的table中挑選沒CA的患者資料，再進行配對

# 先列出chartevent的所有患者
chunk_size = 1000000

column_to_select = 'subject_id'

unique_contents = set()

for chunk in pd.read_csv('chartevents.csv', chunksize=chunk_size, usecols=[column_to_select], dtype={"value": str, "valuenum": str, "valueuom" : str}):
    unique_contents.update(chunk[column_to_select].unique())

chartevent = pd.DataFrame(list(unique_contents), columns=[column_to_select])

chartevent

,subject_id
0,11272192
1,13762560
2,14811141
3,18874374
4,19267596
...,...
50037,11272182
50038,13762552
50039,10747898
50040,13500411


In [ ]:
print(chartevent['subject_id'].nunique())

50042


In [ ]:
# 從noCA的table中抓出chartevent的患者資料，包含年紀和性別
merged_df = noCA.merge(chartevent, on='subject_id', how='inner')
noCA1 = pd.DataFrame(merged_df)
noCA1

,subject_id,gender,anchor_age
0,10000032,F,52
1,10000980,F,73
2,10001217,F,55
3,10001725,F,46
4,10001884,F,68
...,...,...,...
46531,19999297,M,42
46532,19999442,M,41
46533,19999625,M,81
46534,19999840,M,58


In [ ]:
# 配對年齡和性別相同的nonCA患者
pair_noCA = pd.DataFrame()

# 印出CA的每一row
for index, row in CA.iterrows():
    age = row['anchor_age']
    gender = row['gender']

    # 在noCA1中配對年齡和性別相同的患者
    match_rows = noCA1[(noCA1['anchor_age'] == age) & (noCA1['gender'] == gender)]

    if not match_rows.empty:
        # 在noCA1中隨機選擇一位
        selected_row = match_rows.sample(n=1)

        # 將配對到的患者添加到新的dataframe中
        pair_noCA = pair_noCA.append(selected_row)

pair_noCA

,subject_id,gender,anchor_age
26683,15733157,F,74
8633,11851424,M,87
11382,12441371,M,72
30108,16467939,M,41
37894,18154666,F,35
...,...,...,...
29840,16409696,M,56
31736,16817512,M,71
19399,14175762,F,73
12916,12764579,M,56


In [ ]:
print(pair_noCA['subject_id'].nunique())

477


In [ ]:
# 從chartevent中抓出這些患者的vital signs
noCApatient = pd.DataFrame()

chunk_size = 100000

for chunk in pd.read_csv('chartevents.csv', chunksize=chunk_size, dtype={"value": str, "valuenum": str, "valueuom" : str}):
    matched_rows = chunk[chunk['subject_id'].isin(pair_noCA['subject_id'])]

    if not matched_rows.empty:
        noCApatient = pd.concat([noCApatient, matched_rows], ignore_index=True)

noCApatient

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 19:07:00,220045,121,121,bpm,0
1,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 18:53:00,220046,120,120,bpm,0
2,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 18:53:00,220047,50,50,bpm,0
3,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 19:07:00,220210,30,30,insp/min,0
4,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 19:07:00,220277,95,95,%,0
...,...,...,...,...,...,...,...,...,...,...
2676980,19807025,21434336,32771680,2117-10-30 12:53:00,2117-10-30 12:54:00,229138,Vital signs and CIWA / CINA as indicated per p...,NaN,NaN,0
2676981,19807025,21434336,32771680,2117-10-30 12:53:00,2117-10-30 12:54:00,229140,No,NaN,NaN,0
2676982,19807025,21434336,32771680,2117-10-30 12:53:00,2117-10-30 12:54:00,229141,No,NaN,NaN,0
2676983,19807025,21434336,32771680,2117-10-30 13:38:00,2117-10-30 13:41:00,223758,Full code,NaN,NaN,0


In [ ]:
print(noCApatient['subject_id'].nunique())
print(noCApatient['stay_id'].nunique())

477
654


In [ ]:
# 一個subject_id可能會對應兩個以上的stay_id，只需保留第一個stay_id所對應的資料

first_stay_df = noCApatient.groupby('subject_id').first().reset_index()
new_noCApatient = noCApatient[noCApatient['stay_id'].isin(first_stay_df['stay_id'])]

new_noCApatient

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 19:07:00,220045,121,121,bpm,0
1,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 18:53:00,220046,120,120,bpm,0
2,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 18:53:00,220047,50,50,bpm,0
3,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 19:07:00,220210,30,30,insp/min,0
4,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 19:07:00,220277,95,95,%,0
...,...,...,...,...,...,...,...,...,...,...
2676980,19807025,21434336,32771680,2117-10-30 12:53:00,2117-10-30 12:54:00,229138,Vital signs and CIWA / CINA as indicated per p...,NaN,NaN,0
2676981,19807025,21434336,32771680,2117-10-30 12:53:00,2117-10-30 12:54:00,229140,No,NaN,NaN,0
2676982,19807025,21434336,32771680,2117-10-30 12:53:00,2117-10-30 12:54:00,229141,No,NaN,NaN,0
2676983,19807025,21434336,32771680,2117-10-30 13:38:00,2117-10-30 13:41:00,223758,Full code,NaN,NaN,0


In [ ]:
print(new_noCApatient['subject_id'].nunique())
print(new_noCApatient['stay_id'].nunique())

477
477


In [ ]:
# 選取需要的vital signs
target_itemid = ['220210', '220045', '220277', '220179', '220180', '223761', '223762']
filtered_df = new_noCApatient[new_noCApatient['itemid'].isin(target_itemid)]

filtered_df

,subject_id,hadm_id,stay_id,charttime,storetime,itemid,value,valuenum,valueuom,warning
0,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 19:07:00,220045,121,121,bpm,0
3,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 19:07:00,220210,30,30,insp/min,0
4,10035631,29276678,35275147,2116-02-28 18:52:00,2116-02-28 19:07:00,220277,95,95,%,0
14,10035631,29276678,35275147,2116-02-28 18:53:00,2116-02-28 18:53:00,223761,98.8,98.8,°F,0
15,10035631,29276678,35275147,2116-02-28 18:54:00,2116-02-28 19:07:00,220179,134,134,mmHg,0
...,...,...,...,...,...,...,...,...,...,...
2523663,19807025,21434336,32771680,2117-10-30 11:00:00,2117-10-30 11:13:00,220045,86,86,bpm,0
2523664,19807025,21434336,32771680,2117-10-30 11:00:00,2117-10-30 11:13:00,220179,97,97,mmHg,0
2523665,19807025,21434336,32771680,2117-10-30 11:00:00,2117-10-30 11:13:00,220180,56,56,mmHg,0
2523667,19807025,21434336,32771680,2117-10-30 11:00:00,2117-10-30 11:13:00,220210,16,16,insp/min,0


In [ ]:
print(filtered_df['subject_id'].nunique())
print(filtered_df['stay_id'].nunique())

477
477


In [ ]:
# 選擇和添加需要的欄位
target_columns = ['subject_id', 'stay_id', 'charttime', 'itemid', 'valuenum']
filtered_df = filtered_df[target_columns]

# 將兩個dataframe的index設定為subject_id，並合併
filtered_df = filtered_df.set_index('subject_id')
pair_noCA = pair_noCA.set_index('subject_id')

new_filtered = pd.merge(filtered_df, pair_noCA[['gender', 'anchor_age']], left_index=True, right_index=True, how='inner')
new_filtered

,stay_id,charttime,itemid,valuenum,gender,anchor_age
subject_id,,,,,,
10018845,35674354,2184-10-08 04:21:00,220045,83,M,91
10018845,35674354,2184-10-08 04:21:00,220210,18,M,91
10018845,35674354,2184-10-08 04:22:00,220277,99,M,91
10018845,35674354,2184-10-08 04:23:00,220179,157,M,91
10018845,35674354,2184-10-08 04:23:00,220180,70,M,91
...,...,...,...,...,...,...
19975710,32714049,2129-11-26 03:00:00,220045,79,F,69
19975710,32714049,2129-11-26 03:00:00,220210,14,F,69
19975710,32714049,2129-11-26 03:00:00,220277,94,F,69


In [ ]:
# 重新設定index和column順序
new_df = new_filtered.reset_index()
df = new_df.reindex(columns=['subject_id', 'stay_id', 'gender', 'anchor_age', 'charttime', 'itemid', 'valuenum'])
df

,subject_id,stay_id,gender,anchor_age,charttime,itemid,valuenum
0,10018845,35674354,M,91,2184-10-08 04:21:00,220045,83
1,10018845,35674354,M,91,2184-10-08 04:21:00,220210,18
2,10018845,35674354,M,91,2184-10-08 04:22:00,220277,99
3,10018845,35674354,M,91,2184-10-08 04:23:00,220179,157
4,10018845,35674354,M,91,2184-10-08 04:23:00,220180,70
...,...,...,...,...,...,...,...
187512,19975710,32714049,F,69,2129-11-26 03:00:00,220045,79
187513,19975710,32714049,F,69,2129-11-26 03:00:00,220210,14
187514,19975710,32714049,F,69,2129-11-26 03:00:00,220277,94
187515,19975710,32714049,F,69,2129-11-26 03:01:00,220179,154


In [ ]:
# 將itemid和valuenum轉換成int和float
df['charttime'] = pd.to_datetime(df['charttime'])
df['itemid'] = df['itemid'].astype(int)
df['valuenum'] = df['valuenum'].astype(float)

In [ ]:
# 將vital sign的欄位格式進行轉換，根據itemid將的值添加到對應的column中
for index, row in df.iterrows():
    itemid = row['itemid']
    valuenum = row['valuenum']

    if itemid == 220045:
        df.at[index, 'heart_rate'] = valuenum
    elif itemid == 220179:
        df.at[index, 'sbp_ni'] = valuenum
    elif itemid == 220180:
        df.at[index, 'dbp_ni'] = valuenum
    elif itemid == 220210:
        df.at[index, 'resp_rate'] = valuenum
    elif itemid == 223761:
        df.at[index, 'temperature_f'] = valuenum
    elif itemid == 223762:
        df.at[index, 'temperature_c'] = valuenum
    elif itemid == 220277:
        df.at[index, 'spo2'] = valuenum

df.drop(columns=['itemid', 'valuenum'], inplace=True)
df

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,spo2,sbp_ni,dbp_ni,temperature_f,temperature_c
0,10018845,35674354,M,91,2184-10-08 04:21:00,83.0,NaN,NaN,NaN,NaN,NaN,NaN
1,10018845,35674354,M,91,2184-10-08 04:21:00,NaN,18.0,NaN,NaN,NaN,NaN,NaN
2,10018845,35674354,M,91,2184-10-08 04:22:00,NaN,NaN,99.0,NaN,NaN,NaN,NaN
3,10018845,35674354,M,91,2184-10-08 04:23:00,NaN,NaN,NaN,157.0,NaN,NaN,NaN
4,10018845,35674354,M,91,2184-10-08 04:23:00,NaN,NaN,NaN,NaN,70.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
187512,19975710,32714049,F,69,2129-11-26 03:00:00,79.0,NaN,NaN,NaN,NaN,NaN,NaN
187513,19975710,32714049,F,69,2129-11-26 03:00:00,NaN,14.0,NaN,NaN,NaN,NaN,NaN
187514,19975710,32714049,F,69,2129-11-26 03:00:00,NaN,NaN,94.0,NaN,NaN,NaN,NaN
187515,19975710,32714049,F,69,2129-11-26 03:01:00,NaN,NaN,NaN,154.0,NaN,NaN,NaN


In [ ]:
data = df.groupby(['subject_id', 'stay_id', 'gender', 'anchor_age', 'charttime'])
data

In [ ]:
# 將相同時間的vital sign合併成一行row

result = data.agg({'heart_rate': 'sum', 'resp_rate': 'sum', 'sbp_ni':'sum', 'dbp_ni':'sum', 'temperature_f':'sum',
                    'temperature_c':'sum', 'spo2':'sum'}).reset_index()

result

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,sbp_ni,dbp_ni,temperature_f,temperature_c,spo2
0,10018845,35674354,M,91,2184-10-08 04:21:00,83.0,18.0,0.0,0.0,0.0,0.0,0.0
1,10018845,35674354,M,91,2184-10-08 04:22:00,0.0,0.0,0.0,0.0,0.0,0.0,99.0
2,10018845,35674354,M,91,2184-10-08 04:23:00,0.0,0.0,157.0,70.0,0.0,0.0,0.0
3,10018845,35674354,M,91,2184-10-08 04:31:00,58.0,13.0,0.0,0.0,98.0,0.0,98.0
4,10018845,35674354,M,91,2184-10-08 05:00:00,49.0,11.0,134.0,49.0,0.0,0.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...
55800,19975710,32714049,F,69,2129-11-26 01:01:00,0.0,0.0,151.0,72.0,0.0,0.0,0.0
55801,19975710,32714049,F,69,2129-11-26 02:00:00,72.0,13.0,0.0,0.0,0.0,0.0,96.0
55802,19975710,32714049,F,69,2129-11-26 02:01:00,0.0,0.0,128.0,58.0,0.0,0.0,0.0
55803,19975710,32714049,F,69,2129-11-26 03:00:00,79.0,14.0,0.0,0.0,0.0,0.0,94.0


In [ ]:
print(result['subject_id'].nunique())
print(result['stay_id'].nunique())

477
477


In [ ]:
# 將華氏轉換成攝氏
result['temperature_f'] = (result['temperature_f']-32)/1.8
result['temperature_f'] = result['temperature_f'].apply(lambda x: x if 10 < x < 50 else 0)
result

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,sbp_ni,dbp_ni,temperature_f,temperature_c,spo2
0,10018845,35674354,M,91,2184-10-08 04:21:00,83.0,18.0,0.0,0.0,0.000000,0.0,0.0
1,10018845,35674354,M,91,2184-10-08 04:22:00,0.0,0.0,0.0,0.0,0.000000,0.0,99.0
2,10018845,35674354,M,91,2184-10-08 04:23:00,0.0,0.0,157.0,70.0,0.000000,0.0,0.0
3,10018845,35674354,M,91,2184-10-08 04:31:00,58.0,13.0,0.0,0.0,36.666667,0.0,98.0
4,10018845,35674354,M,91,2184-10-08 05:00:00,49.0,11.0,134.0,49.0,0.000000,0.0,96.0
...,...,...,...,...,...,...,...,...,...,...,...,...
55800,19975710,32714049,F,69,2129-11-26 01:01:00,0.0,0.0,151.0,72.0,0.000000,0.0,0.0
55801,19975710,32714049,F,69,2129-11-26 02:00:00,72.0,13.0,0.0,0.0,0.000000,0.0,96.0
55802,19975710,32714049,F,69,2129-11-26 02:01:00,0.0,0.0,128.0,58.0,0.000000,0.0,0.0
55803,19975710,32714049,F,69,2129-11-26 03:00:00,79.0,14.0,0.0,0.0,0.000000,0.0,94.0


In [ ]:
# 將兩個體溫欄位進行合併(若合併的兩個欄位都有值的話，就將兩個值平均，若只有一欄有值另一欄為0，就直接將值填入新的column)

result['temperature'] = np.where((result['temperature_c'] != 0) & (result['temperature_f'] != 0),
                                  (result['temperature_c'] + result['temperature_f']) / 2,
                                  result['temperature_c'] + result['temperature_f'])

result.drop(columns=['temperature_c', 'temperature_f'], inplace=True)
result

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,sbp_ni,dbp_ni,spo2,temperature
0,10018845,35674354,M,91,2184-10-08 04:21:00,83.0,18.0,0.0,0.0,0.0,0.000000
1,10018845,35674354,M,91,2184-10-08 04:22:00,0.0,0.0,0.0,0.0,99.0,0.000000
2,10018845,35674354,M,91,2184-10-08 04:23:00,0.0,0.0,157.0,70.0,0.0,0.000000
3,10018845,35674354,M,91,2184-10-08 04:31:00,58.0,13.0,0.0,0.0,98.0,36.666667
4,10018845,35674354,M,91,2184-10-08 05:00:00,49.0,11.0,134.0,49.0,96.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
55800,19975710,32714049,F,69,2129-11-26 01:01:00,0.0,0.0,151.0,72.0,0.0,0.000000
55801,19975710,32714049,F,69,2129-11-26 02:00:00,72.0,13.0,0.0,0.0,96.0,0.000000
55802,19975710,32714049,F,69,2129-11-26 02:01:00,0.0,0.0,128.0,58.0,0.0,0.000000
55803,19975710,32714049,F,69,2129-11-26 03:00:00,79.0,14.0,0.0,0.0,94.0,0.000000


In [ ]:
# 將0替換成NaN
result1 = result.replace(0, np.nan)
result1

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,sbp_ni,dbp_ni,spo2,temperature
0,10018845,35674354,M,91,2184-10-08 04:21:00,83.0,18.0,NaN,NaN,NaN,NaN
1,10018845,35674354,M,91,2184-10-08 04:22:00,NaN,NaN,NaN,NaN,99.0,NaN
2,10018845,35674354,M,91,2184-10-08 04:23:00,NaN,NaN,157.0,70.0,NaN,NaN
3,10018845,35674354,M,91,2184-10-08 04:31:00,58.0,13.0,NaN,NaN,98.0,36.666667
4,10018845,35674354,M,91,2184-10-08 05:00:00,49.0,11.0,134.0,49.0,96.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
55800,19975710,32714049,F,69,2129-11-26 01:01:00,NaN,NaN,151.0,72.0,NaN,NaN
55801,19975710,32714049,F,69,2129-11-26 02:00:00,72.0,13.0,NaN,NaN,96.0,NaN
55802,19975710,32714049,F,69,2129-11-26 02:01:00,NaN,NaN,128.0,58.0,NaN,NaN
55803,19975710,32714049,F,69,2129-11-26 03:00:00,79.0,14.0,NaN,NaN,94.0,NaN


In [ ]:
# 移除charttime是NA的row
# 去除六個生命徵象都是NA的row

result1 = result1.dropna(subset=['charttime'])
result1 = result1.dropna(how='all', subset=['heart_rate', 'sbp_ni','dbp_ni','resp_rate','temperature','spo2'])
result1

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,sbp_ni,dbp_ni,spo2,temperature
0,10018845,35674354,M,91,2184-10-08 04:21:00,83.0,18.0,NaN,NaN,NaN,NaN
1,10018845,35674354,M,91,2184-10-08 04:22:00,NaN,NaN,NaN,NaN,99.0,NaN
2,10018845,35674354,M,91,2184-10-08 04:23:00,NaN,NaN,157.0,70.0,NaN,NaN
3,10018845,35674354,M,91,2184-10-08 04:31:00,58.0,13.0,NaN,NaN,98.0,36.666667
4,10018845,35674354,M,91,2184-10-08 05:00:00,49.0,11.0,134.0,49.0,96.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
55800,19975710,32714049,F,69,2129-11-26 01:01:00,NaN,NaN,151.0,72.0,NaN,NaN
55801,19975710,32714049,F,69,2129-11-26 02:00:00,72.0,13.0,NaN,NaN,96.0,NaN
55802,19975710,32714049,F,69,2129-11-26 02:01:00,NaN,NaN,128.0,58.0,NaN,NaN
55803,19975710,32714049,F,69,2129-11-26 03:00:00,79.0,14.0,NaN,NaN,94.0,NaN


In [ ]:
# 匯出csv，raw data for demographics
result1.to_csv('mimic nonCA demo.csv', index=False)

In [ ]:
# 確認資料中是否有重複時間的
duplicate_rows = result1[result1.duplicated(['stay_id', 'charttime'], keep=False)].index
result1.loc[duplicate_rows]

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,sbp_ni,dbp_ni,spo2,temperature


In [ ]:
# 去除outlier
result1['sbp_ni'] = result1['sbp_ni'].apply(lambda x: x if 0 < x < 400 else None)
result1['dbp_ni'] = result1['dbp_ni'].apply(lambda x: x if 0 < x < 300 else None)
result1['heart_rate'] = result1['heart_rate'].apply(lambda x: x if 0 < x < 300 else None)
result1['resp_rate'] = result1['resp_rate'].apply(lambda x: x if 0 < x < 70 else None)
result1['temperature'] = result1['temperature'].apply(lambda x: x if 10 < x < 50 else None)
result1['spo2'] = result1['spo2'].apply(lambda x: x if 0 < x <= 100 else None)

result1

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,sbp_ni,dbp_ni,spo2,temperature
0,10018845,35674354,M,91,2184-10-08 04:21:00,83.0,18.0,NaN,NaN,NaN,NaN
1,10018845,35674354,M,91,2184-10-08 04:22:00,NaN,NaN,NaN,NaN,99.0,NaN
2,10018845,35674354,M,91,2184-10-08 04:23:00,NaN,NaN,157.0,70.0,NaN,NaN
3,10018845,35674354,M,91,2184-10-08 04:31:00,58.0,13.0,NaN,NaN,98.0,36.666667
4,10018845,35674354,M,91,2184-10-08 05:00:00,49.0,11.0,134.0,49.0,96.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
55800,19975710,32714049,F,69,2129-11-26 01:01:00,NaN,NaN,151.0,72.0,NaN,NaN
55801,19975710,32714049,F,69,2129-11-26 02:00:00,72.0,13.0,NaN,NaN,96.0,NaN
55802,19975710,32714049,F,69,2129-11-26 02:01:00,NaN,NaN,128.0,58.0,NaN,NaN
55803,19975710,32714049,F,69,2129-11-26 03:00:00,79.0,14.0,NaN,NaN,94.0,NaN


In [ ]:
# 進行線性內插法
interpolate_df = result1.interpolate(limit_direction = 'both')
interpolate_df

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,sbp_ni,dbp_ni,spo2,temperature
0,10018845,35674354,M,91,2184-10-08 04:21:00,83.000000,18.000000,157.0,70.0,99.0,36.666667
1,10018845,35674354,M,91,2184-10-08 04:22:00,74.666667,16.333333,157.0,70.0,99.0,36.666667
2,10018845,35674354,M,91,2184-10-08 04:23:00,66.333333,14.666667,157.0,70.0,98.5,36.666667
3,10018845,35674354,M,91,2184-10-08 04:31:00,58.000000,13.000000,145.5,59.5,98.0,36.666667
4,10018845,35674354,M,91,2184-10-08 05:00:00,49.000000,11.000000,134.0,49.0,96.0,36.611111
...,...,...,...,...,...,...,...,...,...,...,...
55800,19975710,32714049,F,69,2129-11-26 01:01:00,80.500000,15.500000,151.0,72.0,94.0,36.666667
55801,19975710,32714049,F,69,2129-11-26 02:00:00,72.000000,13.000000,139.5,65.0,96.0,36.666667
55802,19975710,32714049,F,69,2129-11-26 02:01:00,75.500000,13.500000,128.0,58.0,95.0,36.666667
55803,19975710,32714049,F,69,2129-11-26 03:00:00,79.000000,14.000000,141.0,60.0,94.0,36.666667


In [ ]:
# 四捨五入
interpolate_df['heart_rate'] = interpolate_df['heart_rate'].apply(lambda x:round(x,0))
interpolate_df['sbp_ni'] = interpolate_df['sbp_ni'].apply(lambda x:round(x,0))
interpolate_df['dbp_ni'] = interpolate_df['dbp_ni'].apply(lambda x:round(x,0))
interpolate_df['resp_rate'] = interpolate_df['resp_rate'].apply(lambda x:round(x,0))
interpolate_df['temperature'] = interpolate_df['temperature'].apply(lambda x:round(x,1))
interpolate_df['spo2'] = interpolate_df['spo2'].apply(lambda x:round(x,0))

In [ ]:
interpolate_df.head(10)

,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,sbp_ni,dbp_ni,spo2,temperature
0,10018845,35674354,M,91,2184-10-08 04:21:00,83.0,18.0,157.0,70.0,99.0,36.7
1,10018845,35674354,M,91,2184-10-08 04:22:00,75.0,16.0,157.0,70.0,99.0,36.7
2,10018845,35674354,M,91,2184-10-08 04:23:00,66.0,15.0,157.0,70.0,98.0,36.7
3,10018845,35674354,M,91,2184-10-08 04:31:00,58.0,13.0,146.0,60.0,98.0,36.7
4,10018845,35674354,M,91,2184-10-08 05:00:00,49.0,11.0,134.0,49.0,96.0,36.6
5,10018845,35674354,M,91,2184-10-08 06:00:00,47.0,13.0,116.0,56.0,96.0,36.6
6,10018845,35674354,M,91,2184-10-08 07:00:00,42.0,14.0,86.0,60.0,97.0,36.5
7,10018845,35674354,M,91,2184-10-08 08:00:00,53.0,15.0,131.0,92.0,96.0,36.4
8,10018845,35674354,M,91,2184-10-08 09:00:00,55.0,15.0,122.0,53.0,94.0,36.4
9,10018845,35674354,M,91,2184-10-08 10:00:00,59.0,11.0,141.0,64.0,96.0,36.4


In [ ]:
# 添加label(negative:0)
interpolate_df.insert(0,'label',0)
interpolate_df

,label,subject_id,stay_id,gender,anchor_age,charttime,heart_rate,resp_rate,sbp_ni,dbp_ni,spo2,temperature
0,0,10018845,35674354,M,91,2184-10-08 04:21:00,83.0,18.0,157.0,70.0,99.0,36.7
1,0,10018845,35674354,M,91,2184-10-08 04:22:00,75.0,16.0,157.0,70.0,99.0,36.7
2,0,10018845,35674354,M,91,2184-10-08 04:23:00,66.0,15.0,157.0,70.0,98.0,36.7
3,0,10018845,35674354,M,91,2184-10-08 04:31:00,58.0,13.0,146.0,60.0,98.0,36.7
4,0,10018845,35674354,M,91,2184-10-08 05:00:00,49.0,11.0,134.0,49.0,96.0,36.6
...,...,...,...,...,...,...,...,...,...,...,...,...
55800,0,19975710,32714049,F,69,2129-11-26 01:01:00,80.0,16.0,151.0,72.0,94.0,36.7
55801,0,19975710,32714049,F,69,2129-11-26 02:00:00,72.0,13.0,140.0,65.0,96.0,36.7
55802,0,19975710,32714049,F,69,2129-11-26 02:01:00,76.0,14.0,128.0,58.0,95.0,36.7
55803,0,19975710,32714049,F,69,2129-11-26 03:00:00,79.0,14.0,141.0,60.0,94.0,36.7


In [ ]:
# 前處理後的資料，for model
interpolate_df.to_csv('mimic nonCA.csv', index=False)